In [54]:
from __future__ import division
import random
import sys
import mido
from mido import Message, MidiFile, MidiTrack, MAX_PITCHWHEEL, MetaMessage
import numpy as np
import math
from music21 import midi
from IPython.display import Image
from scipy.integrate import quad
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from visual_midi import Plotter
from pretty_midi import PrettyMIDI
import magenta.music as mm

In [113]:
import pyfluidsynth

ModuleNotFoundError: No module named 'pyfluidsynth'

> # How to program a beat with Python


Like everything in music, drumbeats are frequencies. More precisely, is a combination of percussions sounding at different frequencies. While pitch frequency is measured as the frequency of the sound wave in hertz (how many times the air molecules vibrates per second), the frequency in drumbeats is best described as how many times a percussion sounds per minute.  Each percussion can have different frequencies or have the same ones but shifted a bit. 

Let's define the tempo as 120 beats per minute, or 500,000 microseconds per beat. I know it is weird, but the midi standard usually requires the tempo in microseconds per beat. (Don't worry about this now, but if you are curious, it is calculated by: 60 seconds in a minute * 1000000 microseconds in a second / 120 beats per minute = 500,000 microseconds in a beat)

For example, let's look at a simple drum beat where the kick has the same frequency as the snare, but they are shifted by one bar. They both will have half the frequency of the tempo, that is, they will sound every other beat. 

In [55]:
outfile = MidiFile()

track = MidiTrack()
outfile.tracks.append(track)


track.append(Message('program_change', program=12))

tempo = 500000
first_frequency = 500000/2
second_frequency = 500000/2
shift = 480

Now we have to convert this to ticks. If they will sound every other beat, and every bit is 480 ticks, they will sound every 960 ticks. 

However, they will be shifted by one bar, so the difference in ticks between them is = frequency in ticks - shift 


In [56]:
delta = 480

In [4]:
for i in range(8):
    track.append(Message('note_on', note=36, velocity=100, channel = 9, time=0))
    track.append(Message('note_on', note=38, velocity=100, channel = 9 ,time=delta))
outfile.save('beat1.mid')

Not bad for a first try! What if we add a snare? And what if we make it interesting by defining its frequency at random, as a frction of the tempo as well as its delta as multiples of a beat (480 ticks)? 



But now we have a problem. We have three different frequencies, so we have to calculate their deltas. As we add more instruments, this gets more and more complicated not so much in the mathematical part, but in the way the midi library is designed, where every sound is added sequentially indicating a delta with respect to the last sound.

For this reason, we divide the midi into a track for each percussion.

The ratio between the percussion sound and the tempo will be the delta between itself in beats. We just multiply that for 480, the ticks per beat to get the delta in a number that we can feed into the MIDI message.


In [57]:
outfile = MidiFile()

track.append(MetaMessage('set_tempo',tempo=500000))
track.append(Message('program_change', program=12))

tempo = 500000
first_frequency = 500000/2
second_frequency = 500000/2
shift = 480

track = MidiTrack()
outfile.tracks.append(track)
track.append(Message('note_on', note=36, velocity=100, channel = 9, time=shift-shift))
for i in range(32):
    track.append(Message('note_on', note=36, velocity=100, channel = 9, time=2*480))
  
track1 = MidiTrack()
outfile.tracks.append(track1)
track1.append(Message('note_on', note=38, velocity=100, channel = 9, time=shift))
for i in range(32):
    track1.append(Message('note_on', note=38, velocity=100, channel = 9, time=2*480))
    
hihat_frequency_ratio = random.randint(1,8)


hihat_shift = 480*random.randint(0,4)
hihat_shift
    
  

track2 = MidiTrack()
outfile.tracks.append(track2)
track2.append(Message('note_on', note=42, velocity=100, channel = 9, time=hihat_shift))
for i in range(32):
    track2.append(Message('note_on', note=42, velocity=100, channel = 9, time=hihat_frequency_ratio*480))

outfile.save('beat2.mid')

Okay, that was very manual. How can we automate that?


# How can we create a beat generator?

First, let's define the tempo that we want

In [58]:
tempo = 100

def tempo_to_microseconds(tempo):
    return 60*1000000/tempo

tempo_ms = tempo_to_microseconds(120)

Now let's create an array with percussion options. For now lets just do kick, snare, hi-hat and a bass drum

In [59]:
# 35 = bass drum, 36 = kick = 36, 38 = snare, 42 = hi hat
percussion_options = [35,36,38,42]

Now let's create a function that takes how many instruments we want to use and returns a tuple with the number of the instrument, a random frequency and a random shift with regards to the start.

In [60]:
def invent_beat_pattern(percussion_options):
    # for now we define from 3 to 8
    output = []
    selected_options = random.choices(percussion_options,k=random.randint(3,10))
    for percussion in selected_options:
        frequency_ratio = np.random.random()*np.random.randint(1,2)
        shift = np.random.randint(0,4)
        output.append((percussion,frequency_ratio,shift))
    return output
        

        

Now let's instanciate a midifile and set its tempo. We have also created the first track 

In [61]:
outfile = MidiFile()
track = MidiTrack()
outfile.tracks.append(track)
track.append(MetaMessage('set_tempo',tempo=500000))
track.append(Message('program_change', program=12))

Now the fun part, fill our midi file with our pattern!

We need to calculate how many sounds are there in 8 bars for each of the selected drum sounds. Since we are taking 8 bars, a common measure for drum patterns, we just need to do a simple proportions rule. For example, is a sound has the same frequency as the tempo, it will sound in every beat. In that case, we would need to add that sound 8 times. However, if the frequency is .5 times that of the tempo, how many times do we have to insert it? Easy, we just multiply .5x8 = 4. We would need to insert it 4 equally spaced times in the 8 bars. If our frequency ratio was .8, we would do .8x8 = 6.4. Since we cannot do 6.4 times, we do six, so we have to take the floor function, with math.floor(). 




In [62]:
def calculate_insert_time(length, ratio):
    '''
    Length in bars, ratio as a float.
    '''
    return length*ratio

In [63]:

def insert_pattern(pattern, bars, tempo):
    midi_file = MidiFile()
    track = MidiTrack()
    midi_file.tracks.append(track)
    # Convert tempo to microseconds per beat
    tempo_in_micros = int(60*1000000/tempo)
    track.append(MetaMessage('set_tempo',tempo=tempo_in_micros))
    track.append(Message('program_change', program=12))
    '''
    Pattern is a list of tuples which contains a midi note, a frequency ratio and a shift.
    Bars is the number of bars in the beat. Recommended is 8, but 16 is also cool.
    Tempo can be given in BPMs
    '''
    print('Pattern: ',pattern)

    for percussion in pattern:

        track = MidiTrack()
        midi_file.tracks.append(track)
        '''
        Calculate the times we are going to insert each one. 
        In the tuples of percussions and frequencies [0] is the note, [1] is the ratio and [2] is the shift
        '''      
        times = calculate_insert_time(bars,percussion[1])
        print('Times: ', times)
        i = 0
        while i < times:
            print('Delta: ', percussion[2]*480)
            track.append(Message('note_on', note=percussion[0], velocity=100, channel = 9, time=percussion[2]*480))
            i += 1
        
    return midi_file
            
    
         
    
    

In [64]:
pattern = invent_beat_pattern(percussion_options)
##file = insert_pattern(pattern,3,120)

In [65]:
pattern

[(36, 0.4649190783757007, 2),
 (42, 0.06973487769183995, 2),
 (42, 0.6670758347240773, 2),
 (35, 0.10920994585005883, 3)]

In [9]:
file.save('beat4.mid')

Well, that beat is terrible. But that was expected, since we just selected everything at random. 

That opens up a fascinating question. What is pleasurable? Why do some rhtyms feel nice, while others don't? 

Each beat is a combination of percussions that sound at different and sometimes equal moments. Each different percussion has its own frequency. The relationships between their frequencies are the basis for aesthetic perception. In Western music, note intervals are classified as consonant and dissonant according to the frequency ratios between the played notes. 

Simple ratios are easier to understand. Your brain is able to recognize it clearly and is able to predict it. More complex ratios are harder to read by the brain, because it they are harder to predict (it takes more compute power to tell the individual signals apart). A complicated 11/8 jazz drum pattern is much harder to predict to the average person than the simple 4/4 of a hit pop song.

When we talk about frequencies in beats, we refer to the the times each percussion sounds in a minute. Another thing that is made of frequency is pitch, or the notes you would hear from a guiter strings. Each note is made up of a characteristic frequency measured in Hertz, much faster than that of drums. That means that if we could play a drum incredibly fast, 440 times a second, we could play an A major. 

So if they are, at the end of the day, the same thing but at different frequencies, will the preference for simple ratios remain? Yes, as you can see in the following graph.

<img width="700" height="800" src='Rationale.jpg'><img>



Too little information is not very helpful and too much is overwhelming, but the brain is able to identify when you are sensing an optimal rate of information and rewards you with dopamine so you keep looking, hearing, eating, smelling or touching whatever the source of information. 

But the brain not only takes into account the amount of information that you are receiving. It also takes into account how much effort it took for you to obtain it. It rewards you when it costs you an optimal amount of effort: not too much, not too little. So aethetic creation is a matter of balancing amount of information conveyed and amount of effort required from the observer to obtain it. 


It is a bit better, because now your brain has to predict more different sounds. This is more engaging and your brain rewards that a little more. 

If we add a third sound, a hi-hat, that sounds in every beat, we add yet more complexity but now this starts to feel like an actual beat you could kinda dance or rap to. But if we start adding instruments like crazy, each sounding at different moments, like those crazy contemprary experimental jazz, your brain will have a tougher time predicting the sounds and will not reward you as much to tell you to stop that effort. 

How can we estimate that optimal point of effort and information conveyed. Music theory, which was developed through years of iterations, tell us that simpler relationships or ratios between the elements of a piece, be it visual or musical or whatever, are more pleasant. So let's test it. 

El hecho de que la función de seno, la onda más fundamental, el origen de todas las ondas, tenga una estructura tal que se mida en 4 pasos en el eje de las x y que cada percusión sea lo mismo: una onda en ese eje con cuatro subdivisiones ideales e infinitas no tan ideales. La belleza está en esos puntos.


Complex polyrhythms are closer to random than simple one. Your brain clasifies them as random and produces less dopamine. It is not able to extract information. It is unknown. 

## Heuristics is not all you need (but you do need it)

Drum heuristics: 

The first order heuristic is having this set of rules. The second order is the values for each one. This values can be modified by the user. Also, heuristics are stored inside a list, so they can be changed, removed, added.

- Have a set number of necesary intruments
- Have a set number of optional instruments
- Have a probability for each optional instrument
- Limit frequency ratios 
- Limit shift ratios

Hypothesis: 

Giving agents predefined heuristics and then making them learn from experience generates better melodies than training only. Making models capable of approaching the mean of all songs it was fed with is boring. Giving an agent some initial rules and then let it learn how to use them and even break them is more fun. We will have an agent that composes music. It carries heuristics inside, so we will add that as an attribute. It also has the capacity to learn, so that will be a function. Learning means adjusting the heuristics for now, basically the probability distributions of the probabilities.

Types of patterns

- 4 on the floor

We will create a new structure to store our beats. It will be an Nx16 dimensional matrix with a minimum of N = 3 columns. Each column represents an instrument and each row a sixteenth.

In [66]:
def create_pattern(genre):
    '''
    Receives a genre and returns a beat pattern of that genre
    '''
    # Here we will put the beat pattern as you might see it in a beat machine or piano roll
    beat = np.zeros((16,16))
    # beat[0], the first array, corresponds to the kick
    # beat[1], corresponds to the snare
    # beat[2], corresponds to the hihat
    # the rest are free
    # 35 = bass drum, 36 = kick = 36, 38 = snare, 42 = hi hat
    #we should substract one from these numbers due to Python's zero indexing
    
    
    if genre.lower() == 'hip%hop' or 'hiphop':
        ## this beat has the kick in the: 1,8, 9, 16, or 1a, 4b, 1c 4, d: pretty simetrical
        ## the hihat in the 1,3,5,7,9,11,13,15, every two sixteenth, 
        ## snare in the 5 and the 13. Also simmetrical  
        # kick
        beat[0][np.array([0, 7, 8, 15])] = 36
        # snare
        beat[1][np.array([4, 12])] = 38
        # hihat
        beat[2][np.array([0,2,4,6,8,10,12,14])] = 42
        
    return pd.DataFrame(beat)

beat = create_pattern('hiphop')

beat.index.size

16

In [67]:
def hiphop_groove():
    '''
    Receives a genre and returns a beat pattern of that genre
    '''
    # Here we will put the beat pattern as you might see it in a beat machine or piano roll
    beat = np.zeros((16,16))
    # beat[0], the first array, corresponds to the kick
    # beat[1], corresponds to the snare
    # beat[2], corresponds to the hihat
    # the rest are free
    # 35 = bass drum, 36 = kick = 36, 38 = snare, 42 = hi hat
    #we should substract one from these numbers due to Python's zero indexing
    
    ## this beat has the kick in the: 1,8, 9, 16, or 1a, 4b, 1c 4, d: pretty simetrical
    ## the hihat in the 1,3,5,7,9,11,13,15, every two sixteenth, 
    ## snare in the 5 and the 13. Also simmetrical  
    # kick
    beat[0][np.array([0, 7, 8, 15])] = 36
    # snare
    beat[1][np.array([4, 12])] = 38
    # hihat
    beat[2][np.array([0,2,4,6,8,10,12,14])] = 42
        
    # randomly insert a conga
    beat[3][random.sample(range(16),4)] = 56
    
    return pd.DataFrame(beat)



In [90]:
def insert_pattern(pattern):
    
    # instanciate MidiFile and a MidiTrack. A MidiFile is made of Midi Tracks
    midi_file = MidiFile()
    #track = MidiTrack()
    
    # You append tracks to the midi files' tracks
    #midi_file.tracks.append(track)
    
    # Convert tempo to microseconds per beat
    tempo_in_micros = int(60*1000000/tempo)
    
    # You build tracks by appending messages to them. Messages can set tempos or instruments, like these ones.
    # But they can also add notes at a specific moment in the track as we will see below
    
    #track.append(MetaMessage('set_tempo',tempo=tempo_in_micros))
    #track.append(Message('program_change', program=9))
    
    # each different percussion will have their own track. In that track, 
    
    track = MidiTrack()
    midi_file.tracks.append(track)
    for index, row in pattern.iterrows():
        # check that the row is not all zeros, not empty of percussion sounds
        if row.equals(pd.Series(np.zeros(16))) == False:
            
            # You append tracks to the midi files' tracks
            
            for times in range(2):
                delta = 120
                for beat in row:
                    if beat > 0:
                        track.append(Message('note_on', note=int(beat), velocity=np.random.randint(90,110), channel = 9, time=delta))

                        delta = 120
                    else:
                        delta += 120
           
    return midi_file
            

In [100]:
beat_df = hiphop_groove()

beat_df

hip_groove = insert_pattern(beat_df)

hip_groove

hip_groove.save('hiphop-groove2.mid')
hip_groove.tracks


[<midi track '' 36 messages>]

In [92]:
plotter = Plotter()
plotter.show(pretty,'out.html')

NameError: name 'pretty' is not defined

In [ ]:
playMidi('hiphop-beat.mid')

In [93]:
from magenta.models.melody_rnn import melody_rnn_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music.protobuf import generator_pb2
from magenta.music.protobuf import music_pb2

from magenta.models.drums_rnn import drums_rnn_sequence_generator

mm.notebook_utils.download_bundle("drum_kit_rnn.mag","bundles")
generator_map = drums_rnn_sequence_generator.get_generator_map()

bundle = sequence_generator_bundle.read_bundle_file('bundles/drum_kit_rnn.mag')

mm.notebook_utils.download_bundle("drum_kit_rnn.mag","bundles")
generator = generator_map["drum_kit"](checkpoint = None, bundle = bundle)

generator.initialize()


from magenta.music import constants


# this is the question: if you have 120 quarters per minute, each how many seconds
# will you have a quarter? that is 60 over the qpm. If you have 
# 4 steps per quarter, each how many seconds do you have a step?
qpm = 86
seconds_per_step = 60.0 / qpm / generator.steps_per_quarter

num_steps_per_bar = constants.DEFAULT_STEPS_PER_BAR
seconds_per_bar = num_steps_per_bar*seconds_per_step
seconds_per_bar


    ## this beat has the kick in the: 1,8, 9, 16, or 1a, 4b, 1c 4, d: pretty simetrical
    ## the hihat in the 1,3,5,7,9,11,13,15, every two sixteenth, 
    ## snare in the 5 and the 13. Also simmetrical  

# 35 = bass drum, 36 = kick = 36, 38 = snare, 42 = hi hat
primer_drums = mm.DrumTrack(
    [frozenset(pitches) for pitches in
        [(36, 42),     (), (42,),    (),
         (42,38), (), (42,),    (36,),
         (36,42),  (), (42,),    (),
         (42,38), (), (42,),    (36,),]])

primer_sequence = primer_drums.to_sequence(qpm=90)

primer_start_time = 0
primer_end_time = primer_start_time + seconds_per_bar

num_bars = 3
generation_start_time = primer_end_time 
generation_end_time = generation_start_time + (seconds_per_bar*num_bars)


# The generator interface is common for all models
generator_options = generator_pb2.GeneratorOptions()

# Add a bit of temperature for more flavor
temperature = 1.1
print("Temperature: " + str(temperature))
generator_options.args['temperature'].float_value = temperature

# Defines the generation section
generator_options.generate_sections.add(
    start_time=generation_start_time,
    end_time=generation_end_time)

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from /var/folders/wm/qxpvx1f163j7hkdz4hbsvk380000gn/T/tmpy2d3o0gh/model.ckpt
Temperature: 1.1


start_time: 2.7906976744186047
end_time: 11.162790697674419

In [123]:
sequence = generator.generate(primer_sequence, generator_options)
mm.plot_sequence(sequence)

mm.midi_io.note_sequence_to_midi_file(sequence, 'algorithmic-perreo.mid')



INFO:tensorflow:Beam search yields sequence with log-likelihood: -85.252640 


Loading BokehJS ...

In [120]:
mm.plot_sequence(primer_sequence)
mm.midi_io.note_sequence_to_midi_file(primer_sequence, 'pyperreo.mid')

Loading BokehJS ...

In [105]:
seq.notes

[]

<message note_on channel=9 note=36 velocity=103 time=480>

In [214]:
# We create a class that receives a style, tempo. Usually style defines tempo but 
# for development testing purposes let's pass it

class Composer:
    
    def __init__(self, style, tempo, heuristics):
        # the heuristics is a list
        self.style = style
        self.tempo = tempo
        self.heuristics = heuristics
        
    # for now compose just receives heuristics for drums and melody. They will be functions.
    def compose():
        return
    
    def tempo_to_microseconds():
        return 60*1000000/self.tempo
    


def basic_personal_heuristic():
    percussion_options = [35,36,38,42]
    output = []
    return output

    
def random_pattern_heuristic():
# for now we define from 3 to 8
    percussion_options = [35,36,38,42]
    output = []
    selected_options = random.choices(percussion_options,k=random.randint(3,10))
    for percussion in selected_options:
        frequency_ratio = np.random.random()*np.random.randint(1,2)
        shift = np.random.randint(0,4)
        output.append((percussion,frequency_ratio,shift))
    return output
    
# this needs to receive a list of percussions and a integer k indicating how often we want the kick to sound. If k = 4, 
# it will sound every 4 beats, which would be standard for a 4/4 signature.
# We call that the base frequency. Every other instrument will be generated relativ to it.

'''
def define_frquency(percussion_list, k=4):
    possible_ratios = [3/2,4/3,5/3,5/4,5/6,5/8,8/9,8/15,9/16,15/16]
    freq_perc_tuples = []
    # we will create a tuple with the percussion and its frequency in delta ticks
    for percussion in percussion_list:
        # if its the first percussion, the kick, we assign k delta
        if(percussion_list.index(percussion) == 0):
            freq_perc_tuples.append(percussion,k)
        else:
            # calculate a random change in frequency multiple of k 
            new_freq = np.random.randint(4)
'''


            
def invent_beat_heuristic_1(percussion_options):
    # for now we define from 3 to 8
    possible_ratios = [3/2,4/3,5/3,5/4,5/6,5/8,8/9,8/15,9/16,15/16]
    pattern = []
    pattern.append((36,1,0))
    pattern.append((38,2,0))
    selected_options = random.choices(percussion_options,k=random.randint(1,5))
    for percussion in selected_options:
        frequency_ratio = np.random.choice(possible_ratios)
        shift = 0
        pattern.append((percussion,frequency_ratio,shift))
    return pattern       

    
      

def insert_pattern(pattern, bars, tempo):
    midi_file = MidiFile()
    track = MidiTrack()
    midi_file.tracks.append(track)
    # Convert tempo to microseconds per beat
    tempo_in_micros = int(60*1000000/tempo)
    track.append(MetaMessage('set_tempo',tempo=tempo_in_micros))

    '''
    Pattern is a list of tuples which contains a midi note, a frequency ratio and a shift.
    Bars is the number of bars in the beat. Recommended is 8, but 16 is also cool.
    Tempo can be given in BPMs
    '''
    print('Pattern: ',pattern)

    for percussion in pattern:

        track = MidiTrack()
        midi_file.tracks.append(track)

        '''
        Calculate the times we are going to insert each one. 
        In the tuples of percussions and frequencies [0] is the note, [1] is the ratio and [2] is the shift
        '''      
        times = calculate_insert_time(bars,percussion[1])

        i = 0
        while i < times:
            print(percussion[0])
            print(math.floor((1/percussion[1])*480))

            track.append(Message('note_on', note=percussion[0], velocity=100, channel = 9, time=math.floor((1/percussion[1])*480)))
            print(track)
            i += 1
        
    return midi_file
            
    
         

In [404]:
percussion_options = [35,36,38,42]
pattern = invent_beat_heuristic_1(percussion_options)
midi_file = insert_pattern(pattern, 8, 500000)
midi_file.save('beat_heuristic_1.mid')


TypeError: insert_pattern() takes 1 positional argument but 3 were given

<message note_on channel=9 note=35 velocity=100 time=1706>

P5, P4, M6, M3, m3, m6, M2, m7, m2, M7, TT

j = [2/1]

Unison-frequency ratio 1:1 yields a value of 2

Octave-frequency ratio 2:1 yields a value of 1.5

Perfect 5th-frequency ratio 3:2 yields a value of 0.833

Perfect 4th-frequency ratio 4:3 yields a value of 0.583

Major 6th-frequency ratio 5:3 yields a value of 0.533

Major 3rd-frequency ratio 5:4 yields a value of 0.45

Minor 3rd-frequency ratio 5:6 yields a value of 0.366

Minor 6th-frequency ratio 5:8 yields a value of 0.325

Major 2nd-frequency ratio 8:9 yields a value of 0.236

Major 7th-frequency ratio 8:15 yields a value of 0.192

Minor 7th-frequency ratio 9:16 yields a value of 0.174

Minor 2nd-frequency ratio 15:16 yields a value of 0.129

## Melody

Melody is just more complicated rhtythm. 

In [213]:
def fall_on_beat(midifile):
    '''
        Function to generate deltas that make notes fall on the beat
    ''' 
    ticks_per_beat = midifile.ticks_per_beat
    # Only choose between multiples of the ticks per beat
    delta = random.choice([i*ticks_per_beat//4 for i in range(0,4)])
    return delta
    
    

def calculate_key(midi_note_number, key_type):
    '''
    midi_note_number: based on the MIDI standard
    key_type options:
        major
        minor_natural
    '''
    key = [midi_note_number]
    
    if key_type.lower() == 'major':
        # Major scales are formed by taking the follwing steps = Whole, Whole, Half, Whole, Whole, Whole, Half
        # Where whole steps are two semtitones and Half are one semitone. 
        # Increasing the midi number means an increase of one semitone
        steps = ['W','W','H','W','W','W','H']
        for i in range(len(steps)):
            # If we need to take a whole step, increase by two, otherwise by one
            if steps[i] == 'W':
                midi_note_number = midi_note_number + 2
                key.append(midi_note_number)
            else:
                midi_note_number = midi_note_number + 1
                key.append(midi_note_number)
    elif key_type.lower() == 'minor_natural':
        # Minor scales are formed by taking the follwing steps = Whole, Whole, Half, Whole, Whole, Whole, Half
        # Whole, Half, Whole, Whole, Half, Whole, Whole
        steps = ['W','H', 'W', 'W', 'H' ,'W', 'W']
        for i in range(len(steps)):
            # If we need to take a whole step, increase by two, otherwise by one
            if steps[i] == 'W':
                midi_note_number = midi_note_number + 2
                key.append(midi_note_number)
            else:
                midi_note_number = midi_note_number + 1
                key.append(midi_note_number)     
    return key

## Simple melody creation

MIDI standard

MIDO Library: A command value of 144 signifies a “note on” event, and 128 typically signifies a “note off” event.
Note values are on a range from 0–127, lowest to highest. For example, the lowest note on an 88-key piano has a value of 21, and the highest note is 108. A “middle C” is 60.
Velocity values are also given on a range from 0–127 (softest to loudest). The softest possible “note on” velocity is 1.
A velocity of 0 is sometimes used in conjunction with a command value of 144 (which typically represents “note on”) to indicate a “note off” message, so it’s helpful to check if the given velocity is 0 as an alternate way of interpreting a “note off” message.

In [403]:
key_options = ['major','minor natural']
key_selec = random.choice(key_options)
key = calculate_key(70, key_selec)

outfile = MidiFile()
track = MidiTrack()
outfile.tracks.append(track)

track.append(MetaMessage('set_tempo',tempo=500000))

track.append(Message('program_change', program=12))

for i in range(random.randint(4,12)):
    note = random.choice(key)
    print(note)
    track.append(Message('note_on', note=note, velocity=100, time=fall_on_beat(outfile)))
    #track.append(Message('note_off', note=note, velocity=100, time=2*fall_on_beat(outfile)))

outfile.save('aver.mid')

def playMidi(filename):
  mf = midi.MidiFile()
  mf.open(filename)
  mf.read()
  mf.close()
  s = midi.translate.midiFileToStream(mf)
  s.show('midi')
    
print(outfile.length)
playMidi('aver.mid')

70
70
70
70
70
70
70
70
70
1.375


Si estas en el primero: toma el que sea. Pasa el intervalo, la nota y el paso (1-8), Dist uniforme

En el segundo
Si tomaste abajo de 5, mueve normal centro para el siguiente
Si tomaste 5 o arriba, 

We create a function to determine is moving the indicated interval will result in landing on key, either left or right

In [157]:
def validate_step(note, interval, key_list):
    
    if (note + interval in key_list):
        valid_steps = interval
    elif (note - interval in key_list):
        valid_steps = (-1)*interval
    else:
        valid_steps = 0
    return valid_steps
    

In [165]:

# we require the midi file because that will be global inside the Composer class
def melody_heuristic_normal(midi_file):
    # This heuristics plays sequential notes that result from drawing intervals from normal distribution
    # centered around middle consonance. All notes played for the same amount of time.
    
    # create a track
    track = MidiTrack()
    midi_file.tracks.append(track)
    #Select the key of the song, using a random between 2 octaves, 60-83
    key_midi = np.random.randint(60,84)
    mel_type = random.choice(['major','minor_natural']) 
    # Compute notes in the key
    key = calculate_key(key_midi, mel_type)
    # all intervals are selected from a normal distribution of intervals ordered by consonance
    # ascending order of consonance as per Malmberg CF. The perception of consonance and dissonance. Psychol Monogr. 1918;25:93–133.
    interval_consonance_rank = ['m2','M7','m7','M2','tt','m3','m6','M6','M3','P4','P5','P8']
    jumps = [1,11,10,2,6,3,8,9,4,5,7,12]
    
    # we select a note from the key with a normal distribution centered around the tonic
    note_from_key = key[math.floor(abs(np.random.normal(0)))]
    
    # we will draw intervals from our list with 
    # 
    current_note = note_from_key
    track.append(Message('note_on', note=current_note, velocity=100, time=480))
    for i in range(7):
        
        print('iteration:quit() ',i)
        drawed_interval = jumps[math.floor(abs(np.random.normal(5)))]
        
        print('current_note drawed interval and key: ',current_note,drawed_interval,key)
        valid_steps = 0
        while valid_steps == 0:
            valid_steps = validate_step(current_note,drawed_interval,key)
            if valid_steps != 0:
                print('valid steps',valid_steps)   
                
                current_note = current_note + valid_steps
                track.append(Message('note_on', note=current_note, velocity=100, time=480))
            else:
                drawed_interval = jumps[math.floor(abs(np.random.normal(5)))]
                    

    return midi_file
            
        
        

In [123]:
k = abs(np.random.normal(0))

calculate_key(90,'minor_natural')

[90, 92, 93, 95, 97, 98, 100, 102]

In [166]:
outfile = MidiFile()
melody_heuristic_normal(outfile)

iteration:  0
current_note drawed interval and key:  73 2 [73, 75, 77, 78, 80, 82, 84, 85]
valid steps 2
iteration:  1
current_note drawed interval and key:  75 6 [73, 75, 77, 78, 80, 82, 84, 85]
valid steps 3
iteration:  2
current_note drawed interval and key:  78 6 [73, 75, 77, 78, 80, 82, 84, 85]
valid steps 6
iteration:  3
current_note drawed interval and key:  84 6 [73, 75, 77, 78, 80, 82, 84, 85]
valid steps -6
iteration:  4
current_note drawed interval and key:  78 6 [73, 75, 77, 78, 80, 82, 84, 85]
valid steps 6
iteration:  5
current_note drawed interval and key:  84 6 [73, 75, 77, 78, 80, 82, 84, 85]
valid steps -6
iteration:  6
current_note drawed interval and key:  78 3 [73, 75, 77, 78, 80, 82, 84, 85]
valid steps -3


<midi file None type 1, 1 tracks, 8 messages>

In [167]:
outfile.save('first_heuristic.mid')

In that case, disonant and consonant intervals had a larger probability. Lets try skewing the probability.

In [168]:
# we require the midi file because that will be global inside the Composer class
def melody_heuristic2(midi_file):
    # This heuristics plays sequential notes that result from drawing intervals from normal distribution
    # centered around middle consonance. All notes played for the same amount of time.
    
    # create a track
    track = MidiTrack()
    midi_file.tracks.append(track)
    #Select the key of the song, using a random between 2 octaves, 60-83
    key_midi = np.random.randint(60,84)
    mel_type = random.choice(['major','minor_natural']) 
    # Compute notes in the key
    key = calculate_key(key_midi, mel_type)
    # all intervals are selected from a normal distribution of intervals ordered by consonance
    # ascending order of consonance as per Malmberg CF. The perception of consonance and dissonance. Psychol Monogr. 1918;25:93–133.
    interval_consonance_rank = ['m2','M7','m7','M2','tt','m3','m6','M6','M3','P4','P5','P8']
    jumps = [1,11,10,2,6,3,8,9,4,5,7,12]
    
    # we select a note from the key with a normal distribution centered around the tonic
    note_from_key = key[math.floor(abs(np.random.normal(0)))]
    
    # we will draw intervals from our list with 
    # 
    current_note = note_from_key
    track.append(Message('note_on', note=current_note, velocity=100, time=480))
    for i in range(7):
        
        print('iteration: ',i)
        drawed_interval = jumps[math.floor(abs(np.random.normal(9)))]
        
        print('current_note drawed interval and key: ',current_note,drawed_interval,key)
        valid_steps = 0
        while valid_steps == 0:
            valid_steps = validate_step(current_note,drawed_interval,key)
            if valid_steps != 0:
                print('valid steps',valid_steps)   
                
                current_note = current_note + valid_steps
                track.append(Message('note_on', note=current_note, velocity=100, time=480))
            else:
                drawed_interval = jumps[math.floor(abs(np.random.normal(5)))]
                    

    return midi_file
            

In [ ]:
file = MidiFile()
midi = melody_heuristic2(file)
file.save('heuristic2.midi')

In [ ]:
file = MidiFile()
midi = melody_heuristic2(file)
file.save('heuristic3.midi')

In [ ]:
file = MidiFile()
midi = melody_heuristic2(file)
file.save('heuristic5.midi')

In [ ]:
def wav2RGB(wavelength):
    '''
    This function converts a wavelength to RGB. We will use this to play with ratios in colors. 
    '''
    w = int(wavelength)

    
    if w >= 380 and w < 440:
        R = -(w - 440.) / (440. - 350.)
        G = 0.0
        B = 1.0
    elif w >= 440 and w < 490:
        R = 0.0
        G = (w - 440.) / (490. - 440.)
        B = 1.0
    elif w >= 490 and w < 510:
        R = 0.0
        G = 1.0
        B = -(w - 510.) / (510. - 490.)
    elif w >= 510 and w < 580:
        R = (w - 510.) / (580. - 510.)
        G = 1.0
        B = 0.0
    elif w >= 580 and w < 645:
        R = 1.0
        G = -(w - 645.) / (645. - 580.)
        B = 0.0
    elif w >= 645 and w <= 780:
        R = 1.0
        G = 0.0
        B = 0.0
    else:
        R = 0.0
        G = 0.0
        B = 0.0

    # intensity correction
    if w >= 380 and w < 420:
        SSS = 0.3 + 0.7*(w - 350) / (420 - 350)
    elif w >= 420 and w <= 700:
        SSS = 1.0
    elif w > 700 and w <= 780:
        SSS = 0.3 + 0.7*(780 - w) / (780 - 700)
    else:
        SSS = 0.0
    SSS *= 255

    return [int(SSS*R), int(SSS*G), int(SSS*B)]

## Self notes:

If note X is consonant with Y, is it because it is similar to its harmonics overtones?

Randomness = all events have same probability -> max amount of information, overwhelming

Absolute certainty = one event one probability -> boring, underwhelming

Do dissonant intervals contain more information? 

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4568680/
    
Most accounts of consonance begin with the interpretation of Greek mathematician and philosopher Pythagoras in the sixth century BCE. According to legend, Pythagoras showed that tones generated by plucked strings whose lengths were related by small integer ratios were pleasing. In light of this observation, the Pythagoreans limited permissible tone combinations to the octave (2:1), the perfect fifth (3:2), and the perfect fourth (4:3), ratios that all had spiritual and cosmological significance in Pythagorean philosophy

- The basis for this auditory “roughness” is the physical interaction of sound waves with similar frequencies, whose combination gives rise to alternating periods of constructive and destructive interference. Interference is maximum information

Sensations of tone

https://en.wikipedia.org/wiki/Sensations_of_Tone

https://cecs.anu.edu.au/people/charles-martin

https://pypi.org/project/pycomposer/
    
That learns from MIDI file

My approach learns from feedback



Collaborative generative music site

Add your own heuristics. See how many people like your heuristics. Add your own rules. 

The agent learns what heuristics you like. It is heuristics centered. 
A GAN can be an heuristic, which was learned. Musicians have both types of heuristics, learned and explicit rule based. That is music theory. 

Heuristic GAN 
Have a GAN that generates sequences and discrimintor that evaluates according to the heuristic.
generator learns heuristic but not fixed

Calculate frequency from last step and calculate new frequency from normal distribution



Fourier series to rhythm and check if it matches simple ratios

http://www.thefouriertransform.com/series/fourier.php

How does the brain sound like? Do the frequencies exhibit simple rhythms?

> How to create music for focusing and concentrationg

We need to find the optimal combination of order and chaos in melody, rhythm and harmony, where order is closer to consonance and chaos to dissonance

Generative art is artwork generated partially or completely by an autonomous system

Ten Questions

http://jonmccormack.info/wp-content/uploads/2012/10/TenQuestionsV3.pdf

https://sensilab.monash.edu/people/jon-mccormack/

Research question: what characteristics does preferred sequences have

DrumRNN cannot handle reggaeton!!! Cultural limitations of models

Can Magenta models handle velocity?

Using MusicVae to merge your faavorite songs. 

Create an app that lets users merge their favorite songs. 

Juntar estos dos
https://medium.com/tensorflow/real-time-human-pose-estimation-in-the-browser-with-tensorflow-js-7dd0bc881cd5

https://magenta.tensorflow.org/pianogenie

Hacer una app que te alerte cuando no estés derecho en un parado de manos

Brian Eno: is the contribution of whoever did the code for Logic less than than of the band that created a hit song in it?
Technology as a constraint on enabler for new artistic creation
https://www.youtube.com/watch?v=JUL8kNYmgsA

Art is dependent on technology

It is great to be creative in code because you are inherently limited. Artists often report that when they are limited, they are the most creative.

In [19]:
f = []
f.append(3)
f.append(4)
for i in f:
    print(f.index(i))

0
1


In [76]:
def normalProbabilityDensity(x):
    constant = 1.0 / np.sqrt(2*np.pi)
    return(constant * np.exp((-x**2) / 2.0) )
zoe_percentile, _ = quad(normalProbabilityDensity, np.NINF, 2)
mike_percentile, _ = quad(normalProbabilityDensity, np.NINF, 1.00)
print('Zoe: ', zoe_percentile)
print('Mike: ', mike_percentile)
    

Zoe:  0.9772498680518209
Mike:  0.8413447460685435


In [78]:
zoe_percentile, _ = quad(normalProbabilityDensity, np.NINF, 2)
mike_percentile, _ = quad(normalProbabilityDensity, np.NINF, 1.00)

array([1.33830226e-04, 1.84295302e-04, 2.52138056e-04, 3.42709873e-04,
       4.62784614e-04, 6.20862299e-04, 8.27514755e-04, 1.09577221e-03,
       1.44154732e-03, 1.88408981e-03, 2.44646147e-03, 3.15601632e-03,
       4.04486639e-03, 5.15030799e-03, 6.51517825e-03, 8.18810653e-03,
       1.02236211e-02, 1.26820683e-02, 1.56292995e-02, 1.91360817e-02,
       2.32771927e-02, 2.81301641e-02, 3.37736510e-02, 4.02854146e-02,
       4.77399263e-02, 5.62056185e-02, 6.57418315e-02, 7.63955298e-02,
       8.81978860e-02, 1.01160853e-01, 1.15273870e-01, 1.30500851e-01,
       1.46777638e-01, 1.64010075e-01, 1.82072870e-01, 2.00809396e-01,
       2.20032535e-01, 2.39526659e-01, 2.59050772e-01, 2.78342808e-01,
       2.97125003e-01, 3.15110210e-01, 3.32008980e-01, 3.47537175e-01,
       3.61423830e-01, 3.73418974e-01, 3.83301094e-01, 3.90883931e-01,
       3.96022313e-01, 3.98616779e-01, 3.98616779e-01, 3.96022313e-01,
       3.90883931e-01, 3.83301094e-01, 3.73418974e-01, 3.61423830e-01,
      

https://www.researchgate.net/publication/318041455_Using_Recurrent_Neural_Networks_to_Judge_Fitness_in_Musical_Genetic_Algorithms#pf4
https://www.researchgate.net/publication/2342018_GenJam_A_Genetic_Algorithm_for_Generating_Jazz_Solos

https://distill.pub/2017/aia/

In [218]:
mf = midi.MidiFile()
mf.open('hiphopmidi.mid')
mf.read()
mf.close()

mf.tracks

playMidi(mf)

OSError: [Errno 63] File name too long: "<MidiFile 2 tracks\n  <MidiTrack 0 -- 6 events\n    <MidiEvent DeltaTime, t=0, track=0, channel=None>\n    <MidiEvent SET_TEMPO, t=None, track=0, channel=None, data=b'\\x07\\n\\xe2'>\n    <MidiEvent DeltaTime, t=0, track=0, channel=None>\n    <MidiEvent TIME_SIGNATURE, t=None, track=0, channel=None, data=b'\\x04\\x02\\x18\\x08'>\n    <MidiEvent DeltaTime, t=0, track=0, channel=None>\n    <MidiEvent END_OF_TRACK, t=None, track=0, channel=None, data=b''>\n  >\n  <MidiTrack 1 -- 476 events\n    <MidiEvent DeltaTime, t=0, track=1, channel=None>\n    <MidiEvent SEQUENCE_TRACK_NAME, t=None, track=1, channel=None, data=b'FL Keys 1'>\n    <MidiEvent DeltaTime, t=0, track=1, channel=None>\n    <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=55, velocity=100>\n    <MidiEvent DeltaTime, t=0, track=1, channel=None>\n    <MidiEvent NOTE_ON, t=None, track=1, channel=1, pitch=77, velocity=84>\n    <MidiEvent DeltaTime, t=0, track=1, channel=None>\n    <M